#### Paso 0

Lectura de los datasets:

In [1]:
# Importar las librerías necesarias
import pandas as pd

# Leer los datasets
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col = False, compression = "zip")

#### Paso 1

Realizar la limpieza de los datasets

In [15]:
# Renombrar la columna "default payment next month" a "default".
test_data = test_data.rename(columns={'default payment next month': 'default'})
train_data = train_data.rename(columns={'default payment next month': 'default'})

# Remover la columna "ID".
test_data=test_data.drop(columns=['ID'])
train_data=train_data.drop(columns=['ID'])

# Remover las filas con valores faltantes en el dataset de entrenamiento
train_data = train_data.loc[train_data["MARRIAGE"] != 0]
train_data = train_data.loc[train_data["EDUCATION"] != 0]

# Remover las filas con valores faltantes en el dataset de prueba
test_data = test_data.loc[test_data["MARRIAGE"] != 0]
test_data = test_data.loc[test_data["EDUCATION"] != 0]

# Modificar la columna 'EDUCATION' para valores > 4, agrupar en "others"
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

#### Paso 2

Dividir los datasets en x_train, y_train, x_test, y_tes:

In [16]:
# Conjunto de entrenamiento
x_train = train_data.drop(columns = "default")
y_train = train_data["default"]

# Conjunto de prueba
x_test = test_data.drop(columns = "default")
y_test = test_data["default"]

#### Paso 3

Crear un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:

##### Transformar las variables categoricas usando el método one-hot-encoding:

In [17]:
# Importar las librerias necesarias
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Seleccionar las columnas categóricas y numéricas
categorical_columns = ["SEX", "EDUCATION", "MARRIAGE"]
numerical_columns = [col for col in x_train.columns if col not in categorical_columns]

# Definir el transformador para las variables categóricas (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers = [
        ('cat', OneHotEncoder(), categorical_columns),
        ('scaler', StandardScaler(), numerical_columns)
    ]
)

#### - Descompone la matriz de entrada usando componentes principales.
#### - El pca usa todas las componentes.
#### - Escala la matriz de entrada al intervalo [0, 1].
#### - Selecciona las K columnas mas relevantes de la matrix de entrada.
#### - Ajusta una red neuronal tipo MLP.

In [18]:
# Importar las librerias necesarias
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

# Crear el pipeline con preprocesamiento, PCA, selección de características y clasificador
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ('feature_selection', SelectKBest(score_func = f_classif)),
    ('pca', PCA()),
    ('classifier', MLPClassifier(max_iter = 15000, random_state = 21))
])

In [ ]:
# Importar las librerias necesarias
from sklearn.model_selection import GridSearchCV

# Definir los parámetros que se van a probar en el GridSearch
param_grid = {
        "pca__n_components": [None],
        "feature_selection__k": [20],
        "classifier__hidden_layer_sizes": [(50, 30, 40, 60)],
        "classifier__alpha": [0.26],
        'classifier__learning_rate_init': [0.001]
    }

# Crear el modelo con GridSearch
model = GridSearchCV(
    pipeline,                      # El pipeline que definimos previamente
    param_grid,                    # Los hiperparámetros a probar
    cv = 10,                       # Número de splits para validación cruzada
    scoring = "balanced_accuracy", # Usamos precisión balanceada
    refit = True,                  # Refinar el modelo con el mejor resultado
    n_jobs = -1,                   # Usar todos los núcleos disponibles
    verbose = 2
)

# Ajustar el modelo con el conjunto de entrenamiento
model.fit(x_train, y_train)

In [ ]:
# Ver los mejores parámetros encontrados
print(f"Mejores hiperparámetros: {model.best_params_}")

In [ ]:
# Importar las librerías necesarias
import os
import gzip
import pickle

# Crear la carpeta si no existe
models_dir = '../files/models'
os.makedirs(models_dir, exist_ok = True)

# Definir la ruta donde se guardará el modelo
model_path = "../files/models/model.pkl.gz"

# Guardar el modelo entrenado en un archivo
with gzip.open(model_path, "wb") as model_file:
    pickle.dump(model, model_file)

print(f"Modelo guardado en '{model_path}'")

In [22]:
# import json
# from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

# def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
#     # Hacer predicciones
#     y_train_pred = model.predict(X_train)
#     y_test_pred = model.predict(X_test)

#     # Calcular métricas para el conjunto de entrenamiento
#     metrics_train = {
#         'type': 'metrics',
#         'dataset': 'train',
#         'precision': float(precision_score(y_train, y_train_pred, zero_division=0)),
#         'balanced_accuracy': float(balanced_accuracy_score(y_train, y_train_pred)),
#         'recall': float(recall_score(y_train, y_train_pred, zero_division=0)),
#         'f1_score': float(f1_score(y_train, y_train_pred, zero_division=0))
#     }

#     # Calcular métricas para el conjunto de prueba
#     metrics_test = {
#         'type': 'metrics',
#         'dataset': 'test',
#         'precision': float(precision_score(y_test, y_test_pred, zero_division=0)),
#         'balanced_accuracy':float(balanced_accuracy_score(y_test, y_test_pred)),
#         'recall': float(recall_score(y_test, y_test_pred, zero_division=0)),
#         'f1_score': float(f1_score(y_test, y_test_pred, zero_division=0))
#     }

#     # Crear carpeta si no existe
#     output_dir = '../files/output'
#     os.makedirs(output_dir, exist_ok=True)

#     # Guardar las métricas en un archivo JSON
#     output_path = os.path.join(output_dir, 'metrics.json')
#     with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
#         f.write(json.dumps(metrics_train) + '\n')
#         f.write(json.dumps(metrics_test) + '\n')

In [23]:
# from sklearn.metrics import confusion_matrix
# # Función para calcular las matrices de confusión y guardarlas
# def calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test):
#     # Hacer predicciones
#     y_train_pred = model.predict(X_train)
#     y_test_pred = model.predict(X_test)

#     # Calcular matrices de confusión
#     cm_train = confusion_matrix(y_train, y_train_pred)
#     cm_test = confusion_matrix(y_test, y_test_pred)

#     # Convertir las matrices de confusión en formato JSON
#     def format_confusion_matrix(cm, dataset_type):
#         return {
#             'type': 'cm_matrix',
#             'dataset': dataset_type,
#             'true_0': {
#                 'predicted_0': int(cm[0, 0]),
#                 'predicted_1': int(cm[0, 1])
#             },
#             'true_1': {
#                 'predicted_0': int(cm[1, 0]),
#                 'predicted_1': int(cm[1, 1])
#             }
#         }
#     metrics = [
#         format_confusion_matrix(cm_train, 'train'),
#         format_confusion_matrix(cm_test, 'test')
#     ]

#     # Guardar las matrices de confusión en el mismo archivo JSON
#     output_path = '../files/output/metrics.json'
#     with open(output_path, 'a') as f:  # Usar 'a' para agregar después de las métricas
#         for metric in metrics:
#             f.write(json.dumps(metric) + '\n')

# # Función principal para ejecutar todo
# def main(model, X_train, X_test, y_train, y_test):
#     # Crear el directorio de salida si no existe
#     import os
#     os.makedirs('../files/output', exist_ok=True)

#     # Calcular y guardar las métricas
#     calculate_and_save_metrics(model, X_train, X_test, y_train, y_test)

#     # Calcular y guardar las matrices de confusión
#     calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test)

# # Ejemplo de uso:
# main(model, x_train, x_test, y_train, y_test)

In [24]:
# Importar las librerías necesarias
import json
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

# Hacer predicciones
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Calcular métricas para el conjunto de entrenamiento
metrics_train = {
    'type': 'metrics',
    'dataset': 'train',
    'precision': float(precision_score(y_train, y_train_pred, zero_division = 0)),
    'balanced_accuracy': float(balanced_accuracy_score(y_train, y_train_pred)),
    'recall': float(recall_score(y_train, y_train_pred, zero_division = 0)),
    'f1_score': float(f1_score(y_train, y_train_pred, zero_division = 0))
}

# Calcular métricas para el conjunto de prueba
metrics_test = {
    'type': 'metrics',
    'dataset': 'test',
    'precision': float(precision_score(y_test, y_test_pred, zero_division = 0)),
    'balanced_accuracy': float(balanced_accuracy_score(y_test, y_test_pred)),
    'recall': float(recall_score(y_test, y_test_pred, zero_division = 0)),
    'f1_score': float(f1_score(y_test, y_test_pred, zero_division = 0))
 }

# Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok = True)

# Guardar las métricas en un archivo JSON
output_path = os.path.join(output_dir, 'metrics.json')
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
    f.write(json.dumps(metrics_train) + '\n')
    f.write(json.dumps(metrics_test) + '\n')

In [25]:
# Importar las librerías necesarias
from sklearn.metrics import confusion_matrix

# Calcular matrices de confusión
cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

# Convertir las matrices de confusión en formato JSON
def format_confusion_matrix(cm, dataset_type):
    return {
        'type': 'cm_matrix',
        'dataset': dataset_type,
        'true_0': {
            'predicted_0': int(cm[0, 0]),
            'predicted_1': int(cm[0, 1])
        },
        'true_1': {
            'predicted_0': int(cm[1, 0]),
            'predicted_1': int(cm[1, 1])
        }
    }

metrics = [
    format_confusion_matrix(cm_train, 'train'),
    format_confusion_matrix(cm_test, 'test')
]

# Guardar las matrices de confusión en el mismo archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'a') as f:  # Usar 'a' para agregar después de las métricas
    for metric in metrics:
        f.write(json.dumps(metric) + '\n')

In [ ]:
import json

# Ruta al archivo de salida
output_file_path = '../files/output/metrics.json'

# Abrir el archivo y cargar las métricas
metrics = []
with open(output_file_path, 'r') as json_file:
    for line in json_file:
        metrics.append(json.loads(line))

# Mostrar las métricas cargadas
for metric in metrics:
    print(metric)